In [10]:
import requests
import pandas as pd
import math
import time

def 下載股價資訊(年份,股票代碼):
    # 生成月份列表: 建立一個包含該年份所有月份第一天的字串列表 (例如 "20250101", "20250201" 等)。這些字串用於構建 TWSE API 的 URL。
    月份列表 = [f"{str(年份)}{str(m).zfill(2)}01" for m in range(1, 13)]
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()
        if data["stat"] != "OK":
            continue
        df = pd.DataFrame(data["data"], columns=data["fields"])    
        # 處理收盤價: TWSE 返回的收盤價可能包含逗號 (例如 "1,234.50")。這行程式碼使用 str.replace(",", "") 移除逗號，然後使用 pd.to_numeric() 將其轉換為數值型態。
        # errors="coerce" 會將無法轉換的值設為 NaN (Not a Number)。
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")
        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快
    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)
# 下載資料
股票代碼 = "4916"
年份="2025"
df = 下載股價資訊(年份,股票代碼)

# 計算每日報酬率
daily_returns = df["收盤價"].pct_change().dropna()

# 計算平均每日報酬率
r_mean = daily_returns.mean()

# 計算每日報酬率的標準差 (代表波動性)
r_std = daily_returns.std()

print(f"股票代碼: {股票代碼} ({年份}年)")
print(f"平均每日報酬率: {r_mean:.2%}")
print(f"每日報酬率標準差 (波動性): {r_std:.2%}")

股票代碼: 4916 (2025年)
平均每日報酬率: 0.54%
每日報酬率標準差 (波動性): 3.86%


In [ ]:
"""
HTTP呼叫模式
參考Message API資料，以request呼叫
https://developers.line.biz/en/reference/messaging-api/#send-reply-message
"""
import requests
import pandas as pd
import math
import time
import requests
from flask import Flask, request
def 下載股價資訊(年份,股票代碼):
    月份列表 = [f"{年份}{str(m).zfill(2)}01" for m in range(1, 13)]
    all_data = []
    for date_str in 月份列表:
        url = f"https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={股票代碼}"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        data = response.json()

        if data["stat"] != "OK":
            continue

        df = pd.DataFrame(data["data"], columns=data["fields"])

        # 處理收盤價
        df["收盤價"] = pd.to_numeric(df["收盤價"].str.replace(",", ""), errors="coerce")

        all_data.append(df[["日期", "收盤價"]])
        time.sleep(1)  # 避免請求過快

    return pd.concat(all_data).dropna().sort_values("日期").reset_index(drop=True)
# 初始化 Flask 應用程式
app = Flask(__name__)

# 設定 LINE BOT Token
BOT_TOKEN = "貼上TOKEN"

# 傳送文字訊息函數
def send_text_message(reply_token, text):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + BOT_TOKEN
    }
    
    payload = {
        "replyToken": reply_token,
        "messages": [text]
    }
    
    # 發送 POST 請求至 LINE Messaging API
    response = requests.post(
        "https://api.line.me/v2/bot/message/reply",
        headers=headers,
        json=payload
    )
    return response

# LINE Webhook 入口
@app.route("/", methods=['POST'])
def linebot():
    global 年份
    # 取得使用者傳來的資料
    data = request.get_json()
    print(data)
    
    # 提取 replyToken
    reply_token = data['events'][0]['replyToken']
    if data['events'][0]['message']['text'][0:2] == "年份":
        年份=data['events'][0]['message']['text'][2:6]
        response = send_text_message(reply_token, {
    "type": "text",
    "text": "請再輸入股票代碼XXXX"
})
    elif data['events'][0]['message']['text'][0:4] == "股票代碼":
        股票代碼=data['events'][0]['message']['text'][4:]
        df = 下載股價資訊(年份,股票代碼)
        r_mean=df["收盤價"].pct_change().dropna().mean()        
        stock_data={
    "type": "text",
    "text": f"{r_mean:.2%}"
}
        response = send_text_message(reply_token, stock_data)
    else:
        print("別鬧")
        response = send_text_message(reply_token, {
    "type": "text",
    "text": "無法辨識，請輸入年份XXXX或股票代碼XXXX"
})
    
    if response.status_code == 200:
        return "OK", 200
    else:
        print("發送訊息失敗:", response.status_code, response.text)
        return "Error", 400

# 啟動 Flask 伺服器
if __name__ == "__main__":
    app.run(port=3000)